In [1]:
import pandas as pd
import sqlite3

## Simple Query functions

In [2]:
DBPATH = "full/osm_ruegen.db"

def Q(query):
    """return query result as pd.DataFrame"""
    with sqlite3.connect(DBPATH) as conn:
        df = pd.read_sql_query(query, conn)
    return df

def H(query):
    """Display query result as html string, without index column"""
    print(Q(query).to_html(index=False))

## Create views

- Union of way and node tags.
- restaurant tags
- a join of restaurant attributes

In [3]:
view_query = """
DROP VIEW IF EXISTS tags;
CREATE VIEW tags AS
   SELECT * FROM nodes_tags 
   UNION ALL 
   SELECT * FROM ways_tags;

DROP VIEW IF EXISTS resto_tags;
CREATE VIEW resto_tags AS 
    select id, key, value
    from tags 
    where id in (select id 
                 from tags 
                 where key = 'amenity' 
                 and value in ('restaurant', 'cafe', 'fast_food', 'ice_cream')
                 )
      and key in ('amenity', 'name', 'cuisine', 'city');

DROP VIEW IF EXISTS resto;
CREATE VIEW resto AS 
    select distinct
        am.id as id,
        na.value as name,
        ci.value as city,
        cu.value as cuisine
    from
        resto_tags am
        left join resto_tags na on
            am.id = na.id
            and na.key = 'name'
        left join resto_tags ci on
            am.id = ci.id
            and ci.key = 'city'
        left join resto_tags cu on
            am.id = cu.id
            and cu.key = 'cuisine';
            
"""

with sqlite3.connect(DBPATH) as conn:
    cur = conn.cursor()
    cur.executescript(view_query)
    conn.commit()

## Number of Nodes, Ways, tags

In [4]:
Q("""
SELECT 'nodes' as element, count(*) AS num FROM nodes
UNION
SELECT 'nodes_tags' as element, count(*) AS num FROM nodes_tags
UNION
SELECT 'ways' as element, count(*) AS num FROM ways
UNION
SELECT 'ways_tags' as element, count(*) AS num FROM ways_tags
UNION
SELECT 'ways_nodes' as element, count(*) AS num FROM ways_nodes
""")

,element,num
0,nodes,519104
1,nodes_tags,89588
2,ways,83902
3,ways_nodes,784533
4,ways_tags,224127


## Users

#### unique users

In [5]:
Q("""
SELECT count(user) as num_users
  FROM (SELECT DISTINCT user FROM nodes 
        UNION 
        SELECT DISTINCT user FROM ways) 
""")

,num_users
0,1277


#### Number of users having only one edit

In [6]:
Q("""
SELECT count(*) as num_once
FROM (SELECT user, count(*) AS edits 
        FROM (SELECT user FROM nodes 
              UNION ALL 
              SELECT user FROM ways) 
      GROUP BY user
      HAVING edits = 1
);
""")

,num_once
0,263


#### Top 10 users

In [7]:
Q("""
SELECT user, count(*) AS edits 
  FROM (SELECT user FROM nodes 
        UNION ALL 
        SELECT user FROM ways) 
GROUP BY user 
ORDER BY edits DESC
LIMIT 10;
""")

,user,edits
0,da-sch,150202
1,Geofreund1,81407
2,Schwedenhagen,61250
3,jacobbraeutigam,46374
4,!i!,17791
5,Kiekin,12646
6,nbuettler,10390
7,north,9084
8,changchun_1,8515
9,SunCobalt,7035


## Query example: Cities, sorted by frequency of apprearance

In [8]:
Q("""
SELECT ci.value AS city, COUNT(*) AS num
FROM nodes_tags ci, nodes_tags na
WHERE ci.type = 'addr'
  AND ci.key = 'city'
  AND na.type = 'regular'
  AND na.key = 'name'
  AND ci.id = na.id
GROUP by city
ORDER BY num DESC
LIMIT 10
""")

,city,num
0,Binz,151
1,Sassnitz,91
2,Bergen auf Rügen,60
3,Stralsund,52
4,Sellin,49
5,Hiddensee,25
6,Baabe,23
7,Mönchgut,21
8,Putbus,21
9,Breege,17


In [9]:
Q("""
SELECT ci.value AS city, COUNT(*) AS num
FROM tags ci, tags na
WHERE ci.type = 'addr'
  AND ci.key = 'city'
  AND na.type = 'regular'
  AND na.key = 'name'
  AND ci.id = na.id
GROUP by city
ORDER BY num DESC
LIMIT 10
""")

,city,num
0,Binz,379
1,Sassnitz,156
2,Sellin,130
3,Bergen auf Rügen,116
4,Stralsund,101
5,Hiddensee,99
6,Mönchgut,88
7,Baabe,60
8,Glowe,57
9,Breege,49


## Restaurants, Cafés etc.

- problem 1: both on nodes and ways
- solution: combied table / view

- problem 2: i want name, place, and cuisine style.
- for most resaurants, not all info is present

- problem 3: several cuisines, separated by semikolon


#### some statistics

In [10]:
Q("""
select count(*) num_in_nodes
from nodes_tags 
where key = 'amenity' 
and value in ('restaurant', 'cafe', 'fast_food', 'ice_cream')
""")

,num_in_nodes
0,524


In [11]:
Q("""
select count(*) num_in_ways
from ways_tags 
where key = 'amenity' 
and value in ('restaurant', 'cafe', 'fast_food', 'ice_cream')
""")

,num_in_ways
0,130


In [12]:
Q("""
select count(*) 
from tags 
where key = 'amenity' 
and value in ('restaurant', 'cafe', 'fast_food', 'ice_cream')
""")

,count(*)
0,654


In [13]:
Q("""select count(distinct id) as total from resto_tags""")

,total
0,654


In [14]:
Q("""
select 'amenity' as key, count(*) as n from resto_tags where key='amenity'
union
select 'city' as key, count(*) as n from resto_tags where key='city'
union
select 'name' as key, count(*) as n from resto_tags where key='name'
union
select 'cuisine' as key, count(*) as n from resto_tags where key='cuisine'
""")

,key,n
0,amenity,654
1,city,275
2,cuisine,300
3,name,612


### joined table

In [15]:
Q("""
select name, city, cuisine 
from resto
order by city
""")

,name,city,cuisine
0,Venezia,None,italian
1,Schwalbennest,None,None
2,Bernstein,None,None
3,"""Zur Sandbank""",None,regional
4,Hitthim,None,None
...,...,...,...
649,Cafė Ummanz,Ummanz,coffee_shop;cake
650,Hiddenseer Fischerklause,Vitte,None
651,Zur Wittower Fähre,Wiek,None
652,Blumencafe,Wiek,coffee_shop


### Where are most restaurants located?

In [16]:
Q("""
select city, count(*) as num 
from resto
group by city
order by num desc
limit 10
""")

,city,num
0,None,379
1,Binz,50
2,Sassnitz,29
3,Sellin,22
4,Stralsund,20
5,Baabe,16
6,Hiddensee,16
7,Mönchgut,15
8,Putbus,13
9,Glowe,12


### Most common cuisine?

In [17]:
Q("""
select cuisine, count(*) as num
from resto
group by cuisine
order by num desc
limit 8
""")

,cuisine,num
0,None,354
1,regional,79
2,fish,39
3,german,28
4,italian,22
5,kebab,15
6,ice_cream,14
7,german;regional,9


## Population

- actual population: Wikipedia has two figures: 62900 or 77000. Not clear which value is correct.
- https://de.wikipedia.org/wiki/Rügen
- Database result is of the same order of magnitude an in between the wikipedia data -> plausible.

In [18]:
Q("""
select sum(po.value) as total_population
from nodes_tags po
     join nodes_tags na using(id)
     join nodes_tags pl using(id)
where po.key = 'population'
  and po.type = 'regular' --> important because there are also OpenGeoDB entries!!!
  and na.key = 'name'
  and na.type = 'regular'
  and pl.key = 'place'
  and pl.type = 'regular'
""")


,total_population
0,69620


In [19]:
Q("""
select na.value as city, 1*po.value as population
from nodes_tags po
     join nodes_tags na using(id)
     join nodes_tags pl using(id)
where po.key = 'population'
  and po.type = 'regular' --> important because there are also OpenGeoDB entries!!!
  and na.key = 'name'
  and na.type = 'regular'
  and pl.key = 'place'
  and pl.type = 'regular'
order by population desc
limit 10
""")

,city,population
0,Bergen auf Rügen,14328
1,Sassnitz,9481
2,Ostseebad Binz,5595
3,Putbus,4330
4,Sellin,2540
5,Sagard,2512
6,Garz,2220
7,Samtens,2209
8,Dranske,1677
9,Brandshagen,1308
